In [1]:
import pandas as pd
import numpy as np
import os
import copy
import math

In [2]:
output_directory = '/Users/francescmartiescofet/Documents/ETH/IVT/eqasim-java/switzerland/SA_scenario_timedata'
last_iter = 5

In [3]:
drt_legs = []
global_avg = []
global_moving_avg = []
global_successive_avg = []
zonal_avg = []
zonal_moving_avg = []
zonal_successive_avg = []
drt_trips_stats = []
for i in range(last_iter + 1):
    iter_path = os.path.join(output_directory,'testDrtZones','ITERS','it.' + str(i)) + \
                '/' + str(i) + '.'
    global_avg.append(pd.read_csv(iter_path + 'drt_travelTimeData_global.csv', sep=';'))
    global_moving_avg.append(pd.read_csv(iter_path + 'drt_travelTimeData_moving.csv', sep=';'))
    global_successive_avg.append(pd.read_csv(iter_path + 'drt_travelTimeData_successive.csv', sep=';'))
    #zonal_avg.append(pd.read_csv(iter_path + 'DrtWaitTimesZonalAvg.csv', sep=';'))
    #zonal_moving_avg.append(pd.read_csv(iter_path + 'DrtWaitTimesZonalMovingAvg.csv', sep=';'))
    #zonal_successive_avg.append(pd.read_csv(iter_path + 'DrtWaitTimesZonalSuccessiveAvg.csv', sep=';'))
    drt_legs.append(pd.read_csv(iter_path + 'drt_legs_drt.csv', sep=';'))
    drt_trips_stats.append(pd.read_csv(iter_path + 'drt_drtTripsStats.csv', sep=','))

link2zones_csv = pd.read_csv(os.path.join(output_directory,'testDrtZones','drt_WayneCountyLink2Zones.csv'), sep=';')
link2zones_csv = link2zones_csv.set_index('link_id')

In [4]:
link2zones = {}
for link_id, zone in link2zones_csv.iterrows():
    link2zones[link_id] = zone.zone.item()

In [5]:
def check_global_avg(global_avg, drt_legs, drt_trips_stats):
    for it, (it_global_avg, it_drt_legs, it_drt_trips_stats) in enumerate(zip(global_avg, drt_legs, drt_trips_stats)):
        print('Iteration ' + str(it))
        print('Global avg: ')
        display(it_global_avg)
        print('Waiting time average from legs: ' + str(it_drt_legs.waitTime.mean()))
        print('Delay factor average from legs: ' + str(np.float64(it_drt_trips_stats.totalTravelTime.sum()) / it_drt_trips_stats.routerUnsharedTime.sum()))
        print('\n')

In [6]:
check_global_avg(global_avg, drt_legs, drt_trips_stats)

Iteration 0
Global avg: 


,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,delayFactor,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Waiting time average from legs: nan
Delay factor average from legs: nan


Iteration 1
Global avg: 


/var/folders/fn/hz4k_mmj3s5dh0mg4g1_vjq80000gn/T/ipykernel_53476/3032094961.py:7: RuntimeWarning: invalid value encountered in double_scalars
  print('Delay factor average from legs: ' + str(np.float64(it_drt_trips_stats.totalTravelTime.sum()) / it_drt_trips_stats.routerUnsharedTime.sum()))


,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,delayFactor,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0


...

Waiting time average from legs: 76.0
Delay factor average from legs: 1.3877990033691072


Iteration 4
Global avg: 


,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,72.200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,delayFactor,1.280363,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Waiting time average from legs: 72.2
Delay factor average from legs: 1.280363465805277


Iteration 5
Global avg: 


,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,78.45000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,delayFactor,1.26009,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Waiting time average from legs: 78.45
Delay factor average from legs: 1.2600901470888988




In [7]:
def check_global_moving_window_avg(global_moving_avg, drt_legs, moving_window, drt_trips_stats):
    for it, it_global_moving_avg in enumerate(global_moving_avg):
        start = 0
        if it >= moving_window:
            start = it - moving_window + 1
        it_drt_legs = pd.concat([drt_legs[j] for j in range(start, it+1)])
        it_drt_trips_stats = pd.concat([drt_trips_stats[j] for j in range(start, it+1)])
        print('Iteration ' + str(it))
        print('Global moving avg: ')
        display(it_global_moving_avg)
        print('Wait time average from legs: ' + str(it_drt_legs.waitTime.mean()))
        print('Delay factor average from legs: ' + str(np.float64(it_drt_trips_stats.totalTravelTime.sum()) / it_drt_trips_stats.routerUnsharedTime.sum()))

        print('\n')
        

In [8]:
check_global_moving_window_avg(global_moving_avg, drt_legs, 5, drt_trips_stats)

Iteration 0
Global moving avg: 


,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,delayFactor,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Wait time average from legs: nan
Delay factor average from legs: nan


Iteration 1
Global moving avg: 


/var/folders/fn/hz4k_mmj3s5dh0mg4g1_vjq80000gn/T/ipykernel_53476/585497598.py:12: RuntimeWarning: invalid value encountered in double_scalars
  print('Delay factor average from legs: ' + str(np.float64(it_drt_trips_stats.totalTravelTime.sum()) / it_drt_trips_stats.routerUnsharedTime.sum()))


,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,delayFactor,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0


...

Wait time average from legs: 76.66666666666667
Delay factor average from legs: 1.3268154093140885


Iteration 4
Global moving avg: 


,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,74.185185,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,delayFactor,1.297032,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Wait time average from legs: 74.18518518518519
Delay factor average from legs: 1.2970318530811693


Iteration 5
Global moving avg: 


,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,76.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,delayFactor,1.279826,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Wait time average from legs: 76.0
Delay factor average from legs: 1.2798264472800625




In [9]:
def check_global_successive_avg(global_successive_avg, drt_legs, weight, drt_trips_stats):
    iters_wait_time_avg = []
    for it_drt_leg in drt_legs:
        iters_wait_time_avg.append(it_drt_leg.waitTime.mean())
    print(iters_wait_time_avg)
    iters_wait_time_successive_avg = []
    for it in range(len(iters_wait_time_avg)):
        if it == 0:
            iters_wait_time_successive_avg.append(iters_wait_time_avg[it])
            continue
        previous_avg = iters_wait_time_successive_avg[it-1]
        if np.isnan(previous_avg):
            iters_wait_time_successive_avg.append(iters_wait_time_avg[it])
            continue
        if np.isnan(iters_wait_time_avg[it]):
            iters_wait_time_successive_avg.append(previous_avg)
            continue
        iters_wait_time_successive_avg.append((1-weight) * previous_avg + weight * iters_wait_time_avg[it])
    
    iters_delay_factor_avg = []
    for it_drt_trips_stats in drt_trips_stats:
        iters_delay_factor_avg.append(np.float64(it_drt_trips_stats.totalTravelTime.sum()) / it_drt_trips_stats.routerUnsharedTime.sum())
    print(iters_delay_factor_avg)
    iters_delay_factor_successive_avg = []
    for it in range(len(iters_delay_factor_avg)):
        if it == 0:
            iters_delay_factor_successive_avg.append(iters_delay_factor_avg[it])
            continue
        previous_avg = iters_delay_factor_successive_avg[it-1]
        if np.isnan(previous_avg):
            iters_delay_factor_successive_avg.append(iters_delay_factor_avg[it])
            continue
        if np.isnan(iters_delay_factor_avg[it]):
            iters_delay_factor_successive_avg.append(previous_avg)
            continue
        iters_delay_factor_successive_avg.append((1-weight) * previous_avg + weight * iters_delay_factor_avg[it])
    
            
    for it, it_global_successive_avg in enumerate(global_successive_avg):
        print('Iteration ' + str(it))
        print('Global successive avg: ')
        display(it_global_successive_avg)
        print('Wait time average from legs: ' + str(iters_wait_time_successive_avg[it]))
        print('Delay factor average from legs: ' + str(iters_delay_factor_successive_avg[it]))
        print('\n')

In [10]:
check_global_successive_avg(global_successive_avg, drt_legs, 0.5, drt_trips_stats)

[nan, nan, 78.66666666666667, 76.0, 72.2, 78.45]
[nan, nan, 1.1538585077916352, 1.3877990033691072, 1.280363465805277, 1.2600901470888988]
Iteration 0
Global successive avg: 


/var/folders/fn/hz4k_mmj3s5dh0mg4g1_vjq80000gn/T/ipykernel_53476/2650775795.py:22: RuntimeWarning: invalid value encountered in double_scalars
  iters_delay_factor_avg.append(np.float64(it_drt_trips_stats.totalTravelTime.sum()) / it_drt_trips_stats.routerUnsharedTime.sum())


,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,delayFactor,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Wait time average from legs: nan
Delay factor average from legs: nan


Iteration 1
Global successive avg: 


,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,delayFactor,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0


...

Wait time average from legs: 77.33333333333334
Delay factor average from legs: 1.2708287555803712


Iteration 4
Global successive avg: 


,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,74.766667,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,delayFactor,1.275596,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Wait time average from legs: 74.76666666666668
Delay factor average from legs: 1.275596110692824


Iteration 5
Global successive avg: 


,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,76.608333,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,delayFactor,1.267843,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Wait time average from legs: 76.60833333333335
Delay factor average from legs: 1.2678431288908614


